# Process UJC cas dataset

In [66]:
from datasets import load_dataset


dataset = load_dataset("ctu-aic/questions_ujc_cas_cs", split="train")

README.md:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/7.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12363 [00:00<?, ? examples/s]

In [67]:
dataset[377]

{'question': 'Jakou doporučujete příručku pro danou problematiku?',
 'specific_question': 'Kde najdu informace o užívání tří teček?',
 'keyword': 'tři tečky',
 'answer': 'Doporučujeme vám Internetovou jazykovou příručku. Výklad věnující se užívání tří teček najdete v kapitole o interpunkci, konkrétní oddíl je zde: https://prirucka.ujc.cas.cz/?id=166.',
 'last_usage': Timestamp('2023-01-09 00:00:00'),
 'variants': None,
 'id': 377,
 'conversations': [{'content': 'Kde najdu informace o užívání tří teček?',
   'role': 'user'},
  {'content': 'Doporučujeme vám Internetovou jazykovou příručku. Výklad věnující se užívání tří teček najdete v kapitole o interpunkci, konkrétní oddíl je zde: https://prirucka.ujc.cas.cz/?id=166.',
   'role': 'assistant'}]}

In [68]:
import re
import nltk
nltk.download('punkt_tab')

def remove_url(text:str) -> str:
    url_in_brackets_pattern = re.compile(r'\s*\([^)(]*https?:\/\/[^\s()]+[^)(]*\)')
    if "http" in text:
        #remove urls in brackets
        text = re.sub(url_in_brackets_pattern, '', text)

        #if url still in the answer
        if "http" in text:
            #handle standalone urls
            ##obtain sentences
            sentences=nltk.tokenize.sent_tokenize(text, language="czech")
            ##find sentences without urls
            filtered_sentences = [sentence for sentence in sentences if "http" not in sentence]
            
            text = " ".join(filtered_sentences)

    
    return text
    

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mlynatom/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [69]:
from ftfy import fix_text


def ujc_cas_map(example):
    input = example["specific_question"] if example["specific_question"] is not None else example["question"]
    input = fix_text(input.strip())
    input = remove_url(input)
    output = fix_text(example["answer"].strip())
    output = remove_url(output)

    if input == "" or output == "":
        example["conversations"] = []
        return example
    
    user_conversation = {"role": "user", "content": input}
    assistant_conversation = {"role": "assistant", "content": output}

    conversations = [
        user_conversation, assistant_conversation
    ]

    example["conversations"] = conversations

    return example

In [70]:
dataset = dataset.map(ujc_cas_map)

Map:   0%|          | 0/12363 [00:00<?, ? examples/s]

In [71]:
dataset[8710]

{'question': 'Jazykové poradenství ve světě',
 'specific_question': 'Na koho se mohu s žádostí o odpověď na jazykový dotaz obrátit na Slovensku, máte tam nějakou partnerskou organizaci?',
 'keyword': 'jazyková poradna',
 'answer': 'V Bratislavě působí Jazyková poradňa Ľudovíta Štúra SAV, více informací a kontakty naleznete na adrese: https://jazykovaporadna.sme.sk/s/onas/.',
 'last_usage': Timestamp('2017-01-19 00:00:00'),
 'variants': None,
 'id': 8710,
 'conversations': []}

In [72]:
dataset.push_to_hub("ctu-aic/questions_ujc_cas_cs", commit_message="Conversations urls removed")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/questions_ujc_cas_cs/commit/f81b4e85cec7439186b00217dc87853814e83b77', commit_message='Conversations urls removed', commit_description='', oid='f81b4e85cec7439186b00217dc87853814e83b77', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/questions_ujc_cas_cs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/questions_ujc_cas_cs'), pr_revision=None, pr_num=None)

In [73]:
dataset.save_to_disk("/home/mlynatom/master-thesis-repository-tomas-mlynar/datasets/instruction_tuning/ujc_cas/questions_ujc_cas_cs_itformat")

Saving the dataset (0/1 shards):   0%|          | 0/12363 [00:00<?, ? examples/s]